# Discretization scalar example


In this live script we aproximately solve the following simple problem

$$\min\sum_{k=0}^{1} x_{k}^{2}+u_{k}^{2}+g_{2}\left(x_{2}\right)$$

$$\text{s.t. } x_{k+1}=x_{k}+u_{k} \quad k \in\{0,1\}$$

where two different terminal costs will be considered:

$$g_{2}\left(x_{2}\right)=e^{x_{2}}$$

or

$$g_{2}\left(x_{2}\right)=x_{2}^2.$$

When $g_{2}\left(x_{2}\right)=x_{2}^2$ is considered, we can obtain the optimal 
policy and costs-to-go with dynamic programming

$$\begin{array}{rlrl}{u_{0}} & {=-\frac{3}{5} x_{0}} & {} & {u_{1}} & =-\frac{1}{2} x_{1} \\
{J_{0}\left(x_{0}\right)} & {=\frac{8}{5} x_{0}^{2}} & {} & {J_{1}\left(x_{1}\right)} & {=\frac{3}{2} x_{1}^{2}}\end{array}$$

After considering the quantized variables

$$x_k \in \delta \bar{x}_k, u_k \in \delta \bar{u}_k, \text{ where } \bar{x}_k \in \{-N,-N+1,\dots,N-1,N\}$$

$$\bar{u}_k \in \{-M,-M+1,\dots,M-1,M\}$$

we obtain the discrete optimization problem

$$\delta^{2}\left(\sum_{k=0}^{1} \bar{x}_{k}^{2}+\bar{u}_{k}^{2}\right)+g_{2}\left(\delta 
\bar{x}_{2}\right) $$

$$\bar{x}_{k+1}=\bar{x}_{k}+\bar{u}_{k} \quad k \in\{0,1\}$$

The following script solves this discrete optimization problem showing that when $g_{2}\left(x_{2}\right)=x_{2}^2$ the optimal policy and costs-to-go can be very well approximated, indicating that this is also the case when $g_{2}\left(x_{2}\right)=e^{x_{2}}$. This latter case can also be tested by changing the flag `optterminalcost`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl

In [ ]:
# parameters defining the problem
h      = 2
N      = 100
M      = 50
delta  = 0.05
x      = np.arange(-N, N+1)*delta
u      = np.arange(-M, M+1)*delta
x_     = N*delta
u_     = M*delta
optterminalcost = 2 # 1 -x^2, 2-exp(x)

# cost function of the discretized system for state i and action j
MM = np.zeros((2*N+1,2*M+1))
for i in range(2*N+1):
    for j in range(2*M+1):
        y = x[i] + u[j]
        if y > x_ or y < -x_:
            MM[i,j] = np.inf
        else:
            MM[i,j] = x[i]**2 + u[j]**2

# terminal cost
J = np.zeros((2*N+1,h+1))
for i in range(2*N+1):
    if optterminalcost == 1:
        J[i,h] = x[i]**2
    else:
        J[i,h] = np.exp(x[i])
    
# run the dynamic programming algorithm
v = np.zeros(2*M+1)
uoptimal = np.zeros((2*N+1,h))
joptimal = np.zeros((2*N+1,h))
for k in range(h-1,-1,-1): # h:-1:1
    for i in range(2*N+1):
        for j in range(2*M+1):
            y = x[i] + u[j]
            if y > x_ or y < -x_:
                v[j] = np.inf
            else:
                indy = np.round( (x[i] + u[j])/delta )+N
                v[j] = MM[i,j] + J[int(indy) , k+1]

        indu = np.argmin(v)
        J[i,k]  = v[indu]
        uoptimal[i,k] = u[indu]
        joptimal[i,k]  = indu
    

In [ ]:
# plots
if optterminalcost == 1:
    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,J[:,2])
    ax.plot(x,x**2)
    ax.grid(True)
    ax.set_xlabel('x_2')
    ax.set_ylabel('J_2')
    ax.legend(['Discretization', 'Optimal solution'])

    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,J[:,1])
    ax.plot(x,1.5*x**2)
    ax.grid(True)
    ax.set_xlabel('x_1')
    ax.set_ylabel('J_1')
    ax.legend(['Discretization', 'Optimal solution'])

    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,J[:,0])
    ax.plot(x,8/5*x**2)
    ax.grid(True)
    ax.set_xlabel('x_0')
    ax.set_ylabel('J_0')
    ax.set_xlim(-3, 3)
    ax.set_ylim(0, 20)
    ax.legend(['Discretization', 'Optimal solution'])

    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,uoptimal[:,1])
    ax.plot(x,-0.5*x)
    ax.set_xlabel('x_1')
    ax.set_ylabel('u_1')
    ax.grid(True)
    ax.legend(['Discretization', 'Optimal solution'])

    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,uoptimal[:,0])
    ax.plot(x,-3/5*x)
    ax.set_xlabel('x_0')
    ax.set_ylabel('u_0')
    ax.set_xlim(-3, 3)
    ax.set_ylim(-2, 2)
    ax.grid(True)
    ax.legend(['Discretization', 'Optimal solution'])
    
else:
    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,J[:,2])
    ax.grid(True)
    ax.set_xlabel('x_2')
    ax.set_ylabel('J_2')
    ax.legend(['Discretization'])
    
    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,J[:,1])
    ax.grid(True)
    ax.set_xlabel('x_1')
    ax.set_ylabel('J_1')
    ax.legend(['Discretization'])
    
    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,J[:,0])
    ax.grid(True)
    ax.set_xlabel('x_0')
    ax.set_ylabel('J_0')
    ax.set_xlim(-3, 3)
    ax.set_ylim(0, 20)
    ax.legend(['Discretization'])
    
    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,uoptimal[:,1])
    ax.set_xlabel('x_1')
    ax.set_ylabel('u_1')
    ax.grid(True)
    ax.legend(['Discretization'])
    
    f = plt.figure()
    ax = plt.gca()
    ax.stem(x,uoptimal[:,0])
    ax.set_xlabel('x_0')
    ax.set_ylabel('u_0')
    ax.set_xlim(-3, 3)
    ax.set_ylim(-2, 2)
    ax.grid(True)
    ax.legend(['Discretization'])